## Universal Robot mit Objekterkennung auf Fließband

Wir können uns jetzt einem etwas komplexeren Beispiel widmen.

Im nachfolgenden soll ein Universal Robot (UR5) simuliert werden, der mit einer Kamera rote Blöcke auf einem Fließband detektiert, diese mit einem Vakuumgreifer aufnimmt und in einer Kiste ablegt

Dazu werden wir im *Terminal* arbeiten. Um ein neues *Terminal* zu öffnen, könnt ihr die Tastenkombination **STRG** + **ALT** + **T** nutzen.

![](img/Terminal.png)

Wenn ihr mit der *rechten* Maustaste in das Terminal klickt, könnt ihr das Terminal außerdem aufteilen und mehrere Terminals in einem Fenster erzeugen.

Erzeugt folgendes Setup von 4 Terminals in einem Fenster:

![](img/multiterminal.png)


Nun können wir auch die Simulation starten, indem wir den Befehl

**roslaunch ur5_notebook initialize.launch**

eintippt.

![](img/multterm_command.png)

Daraufhin sollte sich auch der Simulator *Gazebo* öffnen

![](img/gazebo_default.png)

Beenden kann man ihn, indem ihr in das entsprechende Terminal klickt und mit **STRG** + **C** den Vorgang abbrecht (kann etwas dauern). Das funktioniert genauso bei allen anderen Programmen, die wir später in den anderen Terminals aufrufen werden!


### Launch Files

Mehrere Programme können geordnet gestartet werden, indem sie in einem .launch file definiert werden.
Das sind üblicherweise komplexere Programme. Hier starten wir zum Beispiel den Simulator, fügen die simulierten Objekt ein, unter anderem einen UR5 Roboter und starten die zugehörigen Regler, Bahnplaner etc.

Wie ihr bemerkt habt gebt ihr zwei Parameter an den *roslaunch* Command:

* *ur5_notebook* ist das Package, also die modulare Einheit, zu der das launch file gehört
* *initialize.launch* ist das Launch file an sich, das sich in dem entsprechenden Unterordner *launch* im Packageordner angelegt wurde

Schaun wir uns einmal die Struktur eines ROS-Packages an. Dazu können wir in einem **neuen** Terminal einmal in den entsprechenden Ordner des Packages wechseln. Das geht mit dem Befehl

**cd $(rospack find ur5_notebook)**

Wie wir bemerken können wir den absoluten Pfad im System mit dem Befehl *rospack find <package_name>* finden. Diesen übergeben wir als Parameter dem Befehl *cd*, mit dem man den Ordner wechseln kann.

Wir können nun den Editor *Atom* öffnen, mit dem wir eine Übersicht über die Ordner und Dateien bekommen und diese auch bearbeiten können. Dazu nutzen wir den Befehl

**atom -n .**

**(Achtung, nicht den Punkt am Ende vergessen!)**

![](img/openatom.png)

Auf der linken Seite von Atom finden wir die Dateien und Ordner, z.B. auch das bereits erwähnte *initialize.launch* im Ordner *launch*.

![](img/atomfiles.png)

Ihr könnt euch das mal ansehen und könnt sehen, dass bei dieser Simulation ein etwas komplexerer Startup-Prozess stattfinded.

So können aus einem launch file andere launchfiles mit Parametern aufgerufen werden. Zum Beispiel wird aus dem Package *gazebo_ros* eine leere Welt geladen und mit dem Simulator *gazebo* in der GUI gestartet.

~~~~
<include file="$(find gazebo_ros)/launch/empty_world.launch">
    <arg name="world_name" default="worlds/empty.world"/>
    <arg name="paused" value="$(arg paused)"/>
    <arg name="gui" value="$(arg gui)"/>
</include>
~~~~

Es lassen sich aber auch Programme direkt starten, wie der *blockspawner*, der die Blöcke in unserer Simulation erzeugt.

~~~~
<node name="blocks_spawner" pkg="ur5_notebook" type="blocks_spawner" output="screen" />
~~~~

Dazu muss man den 
* Package Namen (hier *ur5_notebook*)
* Typ (also wie die ausführbare Datei heißt, die erzeugt wird. Wichtig: Wir programmieren in Python, das heißt wir erzeugen keine neue ausführbare Datei wie beim Blockspawner, der in C++ programmiert ist, sondern lassen unseren Code direkt ausführen. Deswegen steht bei uns an der Stelle des Types immer der komplette Dateiname inklusive Endung z.b. ur5_vision.py)
* Output - wohin die Ausgabe der Datei geleitet wird. Im Normalfall wollen wir es im Terminal anzeigen lassen. Dazu setzen wir den output parameter auf *screen* - so wie hier
* Node Namen, einen eindeutigen Namen, damit wir auf dem System die laufenden Programme eindeutig identifizieren können


## Package XML

Wir wissen ja bereits, dass die Hauptmotivation hinter dem ROS System Modularität ist. Um ein abgeschlossenes Modul zu definieren, müssen wir eine *package.xml* erzeugt haben. Diese findet ihr ebenfalls in Atom im Ordner.

Hier finden wir z.B. den eindeutigen Namen des Pakets, die Version, eventuelle Maintainer und License.

~~~~
<package>
  <name>ur5_notebook</name>
  <version>0.0.0</version>
  <description>The ur5_notebook package</description>
~~~~


Außerdem werden hier auch die Abhängigkeiten zu anderen Packages definiert, die wir zum Ausführen der Programme in dem Package benötigen, hier z.B. *message_generation* weil wir neue Messages für ROS Topics definieren.

Diese zu definieren ist sehr simpel. Unter *msgs/Tracker.msg* findet ihr eine Message die wir noch benötigen werden. Diese besteht einfach nur aus 9 Werten, die mit vorgefertigten Datentypen versehen sind.

~~~~
float64 x  # x coordinate in the world
float64 y  # y coordinate in the world
float64 z  # z coordinate in the world
float64 error_x
float64 error_y
float64 error_z
bool flag1
bool flag2
bool flag3
~~~~

Die meisten anderen Dateien können wir erstmal ignorieren, insbesondere die *CMakeLists.txt* ist noch wichtig für die Erzeugung von ausführbaren Dateien aus den *.cpp* Dateien (die in C++ geschrieben sind).
Diese sind aber im weiteren Verlauf aber unwichtig für uns und dienen nur dazu, die roten Blöcke einzufügen.

Leider bewegt sich noch nichts im Simulator, da die drei wesentlichen Komponenten für den Programmablauf fehlen:

* Objekterkennung - Aus dem Kamerabild muss das Objekt erkannt werden und die Information über die Position über das Topic **cxy** weitergegeben werden.
* Bahnplanung - Der Bahnplaner muss die Information aus dem **cxy** Topic über die Position bekommen und diesen mit dem Roboter folgen. Außerdem muss er den Greifer über das Topic **cxy1** informieren, wenn er die Greifposition erreicht hat.
* Greifer - Die 9 physischen Sauggreifer in der Simulation müssen koordiniert mit einem **Service Call** gestartet werden, wenn auf dem Topic **cxy1** die Information veröffentlicht wird, dass der Roboter in Greifposition ist

![](img/diagram.png)

Jede dieser Funktionen ist in einem eigenen Programm/Datei vordefiniert und lässt sich alleinstehend starten und beenden.

Mit dem Befehl

**rosrun ur5_notebook &lt;Dateiname>**
    
lässt sich in einem der oben vorbereiteten übrigen 3 Terminalfeldern das Programm starten (z.b. *rosrun ur5_notebook ur5_vision.py*)

Es ist

* Objekterkennung - *ur5_vision.py*
* Bahnplanung - *ur5_mp.py*
* Greifer - *ur5_gripper.py*

Wenn ihr diese Programme alle gestartet habt, werdet ihr bemerken, dass leider noch einige Fehler im Programmcode vorhanden sind, die wir im nachfolgenden beheben werden.

### Objekterkennung

Datei: **ur5_vision.py**

Leider ist die Objekterkennung noch nicht ganz funktionstüchtig und muss noch in die ROS Kommunikation angebunden werden.

Die Kamera publiziert bereits ein Bild auf dem Topic /ur5/usbcam/image_raw, das die Objekterkennung nutzen soll, um die Position des roten Blocks zu bestimmen.

Diese Funktion ist bereits implementiert und heißt *image_callback*. 

Es muss nun ein *Subscriber* definiert werden, der das Bild an diese Funktion weiterleitet.

**Achtung**

Im folgenden werden wir mit sogenannten *Klassen* arbeiten. Bei dieser wird am Anfang einmal die Funktion **__init__** aufgerufen. Innerhalb dieser sollet ihr auch die Publisher und Subscriber definieren. Damit ihr aber von außerhalb darauf zugreifen könnt, z.b. in der Funktion **image_callback**, musst ihr **self.** davor setzen. Umgekehrt gilt das natürlich genauso, wollt ihr z.B. auf **image_callback** in **__init__** zugreifen, müsst ihr **self.image_callback** nutzen.


Außerdem fehlt im Anschluss der *Publisher*, der die Informationen dann auf dem Topic cxy weitergibt. (Tipp: das sollte ganz am Ende des **image_callbacks** passieren)

Achtet auch darauf, dass ihr immer den richtigen Typ verwendet (wie ihr den Typ eines Topics herausfindet haben wir in der letzten Übung behandelt).

Startet im Anschluss das Programm, wie oben beschrieben.

**Bonusaufgabe**

Es noch einen anderen Fehler, den ihr im weiteren Verlauf entdecken werdet

In dem Bildverarbeitungsframework lässt sich auch naoch die Information der erkannten Position darstellen.
Findet (mit Google ;) ) heraus, wie ihr mit OpenCV und Python einen Kreis um die Position, einen Text der die Position anzeigt und die Kontur um den roten Block einzeichnet. 

### Bahnplanung

Datei: **ur5_mp.py**

Nach dem Greifen fährt der Roboter noch nicht zurück in seine Ausgangsposition.

Findet heraus, wie der Wegplaner funktioniert (Tipp: ihr findet eine Vorlage unter *if self.track_flag*) und implementiert den Aufruf auf den Wegplaner zurück zur Startposition.
(Der code muss ganz unten unter *else* eingefügt werden.) 

Die Position ist bereits gespeichert in der Variablen *self.idlepose*)

Startet im Anschluss das Programm, wie oben beschrieben.



### Greifer

Datei: **ur5_gripper.py**

Auch die Greifernode ist noch gar nicht fertig.

Hier müssen noch die *Service* Aufrufe implementiert werden. Diese könnt ihr herausfinden, indem ihr euch die Services alle anzeigen lasst und nach verdächtigen Services sucht (vielleicht steht da ja was bezüglich Vaccum Gripper ;))

Die Greifer sollen *on* gesetzt werden, sobald die auf dem *Topic* **cxy1** das trigger flag2 auf *on* steht.

Startet im Anschluss das Programm, wie oben beschrieben.


###  Block_Killer

Datei: **block_killer.py**

Nach einiger Zeit läuft die Simulation leider auch voll mit immer neuen Klötzen.

Implementiert diesmal eure eigene Node, die sich 
* die vorhandene Objekte ausgeben lässt ( mit dem Service /gazebo/get_world_properties ) und 
* überflüssige dann löscht (mit dem Service /gazebo/delete_model )



### Launch file verändern

Datei: **launch/initialize.launch**

Fügt doch die veränderten Dateien direkt in das Launchfile hinzu. Wie das genau geht haben wir ganz am Anfang gesehen.


### RVIZ

Um die Pfadplanung und den genauen Aufbau des Roboters zu sehen, kann man diesen über das ROS eigene Tool *RVIZ* visualisieren. Dieses lässt sich in einem neuen Terminal mit dem Befehl

**rviz**

aufrufen.

![](img/rviz1.png)

Um die Visualisierung zu sehen muss man unter *Global Options*/*Fixed Frame* das richtige Referenzkoordinatensystem auswählen. In unserem Fall ist das *base_link*

![](img/rviz_fixedframe.png)

Jetzt muss noch das Robotermodell eingefügt werden. Gehe dazu unten links auf den Button *Add* und wähle das Robotermodell aus.

![](img/rviz_robotmodel.png)

Außerdem wollen wir die Koordinatensysteme der jeweiligen Gelenke des Roboters visualisieren. Dazu öffnen wir TF.

![](img/rviz_tf.png)


ROS berechnet aufgrund der geometrischen Beschreibung des Roboters mithilfe einer kinematischen Kette (also mehrere Gelenke, die miteinander verbunden sind) die Positionen jedes einzelnen Gelenks, aber auch insbesondere die Position des Endeffektors, also unseres Greifers.

Als nächstes visualisiert noch die Bahnplanung mittels RVIZ








Falls ihr bis hier her gekommen seid, seid ihr schon echte ROS Profis und könnt bald eure eigenen erfolgreichen Roboteranwendungen umsetzen!

![](img/robot.gif)

![](img/robot.gif)